In [0]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score, \
                            precision_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
import random
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc  
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
rand_seed = 123
random.seed(rand_seed)
np.random.seed(rand_seed)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172.csv')
train = train.drop(['Unnamed: 0','Unnamed: 0.1', 'userid', 'city'], axis = 1)
train.shape

(8965, 208)

In [0]:
# train[train['click'] == 1]['payprice'].value_counts()

In [0]:
validation = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_VALIDATION_v1.csv')
validation = validation.drop(['Unnamed: 0', 'userid', 'city'], axis = 1)
validation.shape

(303925, 208)

In [0]:
x_train = train.drop(['click','bidid'], axis=1)
y_train = pd.DataFrame(train['click'])

In [0]:
x_val = validation.drop(['click','bidid'], axis=1)
y_val = pd.DataFrame(validation['click'])

In [0]:
def classifier(model,  x_train, y_train, x_val, y_val):
        # Fit classifier to training data
    % time model.fit(x_train, y_train)
        # Predict on validation set
    preds = model.predict(x_val)

        # Predicted probabilities
    pred_proba = model.predict_proba(x_val)
    
    pred_proba_click = [p[1] for p in pred_proba]
  
        # Evaluate performance
    print("\n")
    print("Confusion Matrix:")
    print(confusion_matrix(y_val, preds))  #把0分为0 的个数，0-1，1-0，1-1
    print("\n")
    
        # The most important measure is TP/FP rate for the clicked class
    kpi = precision_score(y_val, preds)
    print("Click Precision: {}".format(kpi))
    
    accuracy = accuracy_score(y_val, preds)
    print("\n")
    print("Prediction accuracy: {}".format(accuracy))
    print("\n")
    
    recall = recall_score(y_val, preds)
    print("Recall score: {}".format(recall))
    print("\n")   
    
    roc_auc = roc_auc_score(y_val, preds)
    print("ROC and AUC Score: {}".format(roc_auc))
    print("\n")   
    
    f1 = f1_score(y_val,preds)
    print("f1 score: {}".format(f1))
    print("\n") 
    
    # Store probabilities
    submission = pd.DataFrame(data = list(zip(validation["bidid"], pred_proba_click)),  #validation is the validation dataset
                               columns = ["bidid", "click_proba"])
    
    #submission.to_csv(lr.__doc__.split("(")[0][:-1] + ".csv")
    
    return f1, submission

In [0]:
# def feature_selection(model, x_train, y_train, x_val, rate):  #rate is a number, the smaller, the better, the longer
  
#   stepsize = rate
#   rfecv = RFECV(model, step = stepsize, scoring = 'f1')
#   rfecv = rfecv.fit(x_train, y_train)
  
#   print(rfecv.support_)  #得到列是否被选的逻辑值
#   print("Optimal number of features is {}".format(rfecv.n_features_))

#   selected_features = []

#   for j in range(len(x_train.columns.tolist())):
#     if rfecv.ranking_[j] == 1:
#       selected_features.append(x_train.columns.tolist()[j])
  
#   x_train2 = x_train[selected_features]
#   x_val2 = x_val[selected_features]
  
#   plt.xlabel("Number of features selected (x{})".format(stepsize))
#   plt.ylabel("Cross validation score")
# #   plt.title(str(model))
#   plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
  
#   return x_train2, x_val2, selected_features

In [0]:
def feature_selection_2_fixed_features(model, x_train, y_train, x_val, rate, features):  #rate is a number, the smaller, the better, the longer
  
  stepsize = rate
  rfe = RFE(model, step = stepsize, n_features_to_select = features)
  rfe = rfe.fit(x_train, y_train)
  
  print(rfe.support_)  #得到列是否被选的逻辑值
  print("Optimal number of features is {}".format(rfe.n_features_))

  selected_features = []

  for j in range(len(x_train.columns.tolist())):
    if rfe.ranking_[j] == 1:
      selected_features.append(x_train.columns.tolist()[j])
  
  x_train2 = x_train[selected_features]
  x_val2 = x_val[selected_features]
  
#   plt.xlabel("Number of features selected (x{})".format(stepsize))
#   plt.ylabel("Cross validation score")
# #   plt.title(str(model))
#   plt.plot(range(1, len(rfe.grid_scores_) + 1), rfe.grid_scores_)
  
  return x_train2, x_val2, selected_features

In [0]:
def non_linear(ctr, validation):
  range_all = np.arange(100, 200, 10)
#   range_all = [140]
  lamda = 0.001
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  for c in range_all:
    current_bid = 1.3 * c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))    
#     current_bid =  ((c/lamda) * ctr + c**2) **0.5 - c   #non_linear 2
#     print(current_bid)
    budget = 6250 * 1000
    clicks = 0
    impressions = 0
    
    for j in range(len(current_bid)):
      payprice = validation.payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation.click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
#     print("\n")
    print('C = {}, clicks = {}, impression = {}, budget = {}'.format(c, clicks, impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  c_optimal = range_all[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when c = {}, budget = {}, impression = {}'.format(max_click, c_optimal, budget_max, impression_max))

  return max_click, c_optimal, bid_price_final

In [0]:
# for i in range(20,150, 10):
    
#     x_train_xg, x_val_xg, feature_xg  = feature_selection_2_fixed_features(XGBClassifier(random_state = rand_seed), 
#                                           x_train, y_train, x_val, 10, i)
#     xg_f1, xg_ctr = classifier(XGBClassifier(n_estimators = 500, max_depth = 5, learning_rate = 0.01, random_state = rand_seed), \
#                                              x_train_xg, y_train, x_val_xg, y_val)
#     max_click_n_xg, c_optimal_n_xg, bid_price_final = non_linear(xg_ctr['click_proba'], validation)
    
#     print('feature number is {}. maximum click is {}'.format(i,max_click_n_xg))
    
# # features 90 c 140

In [0]:
x_train_xg, x_val_xg, feature_xg  = feature_selection_2_fixed_features(XGBClassifier(random_state = rand_seed), 
                                          x_train, y_train, x_val, 10, 90)
xg_f1, xg_ctr = classifier(XGBClassifier(n_estimators = 500, max_depth = 5, learning_rate = 0.01, random_state = rand_seed), \
                                             x_train_xg, y_train, x_val_xg, y_val)
max_click_n_xg, c_optimal_n_xg, bid_price_final = non_linear(xg_ctr['click_proba'], validation)
    

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True False False  True False False False
  True False False False  True False  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True False
  True False False  True False False False False False  True False False
 False False False False False False  True  True False  True False False
 False False  True False False  True  True False False False False  True
 False  True  True  True  True  True False False  True False False False
  True  True  True  True False False False  True False False  True  True
 False False False False False False  True  True False False False False
 False False False False False  True False False False False False  True
 False False False False False False False False False False False False
 False False False False False False  True False Fa

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 16.8 s, sys: 43.2 ms, total: 16.9 s
Wall time: 16.9 s


Confusion Matrix:
[[298715   5008]
 [    96    106]]


Click Precision: 0.02072741493938209


Prediction accuracy: 0.9832063831537385


Recall score: 0.5247524752475248


ROC and AUC Score: 0.7541318833930983


f1 score: 0.03987960872836719


C = 100, clicks = 161, impression = 124233, budget = 768348
C = 110, clicks = 163, impression = 124366, budget = 679333
C = 120, clicks = 164, impression = 124221, budget = 620782
C = 130, clicks = 164, impression = 123903, budget = 582896
C = 140, clicks = 166, impression = 123524, budget = 561778
C = 150, clicks = 166, impression = 122997, budget = 552849
C = 160, clicks = 166, impression = 122510, budget = 548201
C = 170, clicks = 166, impression = 121926, budget = 553313
C = 180, clicks = 166, impression = 121294, budget = 566419
C = 190, clicks = 166, impression = 120678, budget = 584697
Maxium is 166, when c = 140, budget = 561778, impression = 123524


In [0]:
# x_train_1 = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172_xg.csv')

In [0]:
# x_train_1 = x_train_1.drop('Unnamed: 0', axis = 1)

In [0]:
# x_val_1 = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/val_1793_7172_xg.csv')

In [0]:
# x_val_1 = x_val_1.drop('Unnamed: 0', axis = 1)

In [0]:
# xg_f1, xg_ctr = classifier(XGBClassifier(n_estimators = 500, max_depth = 5, learning_rate = 0.01, random_state = rand_seed), \
#                                              x_train_1, y_train, x_val_1, y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 5.63 s, sys: 19 ms, total: 5.65 s
Wall time: 5.65 s


Confusion Matrix:
[[298902   4821]
 [    97    105]]


Click Precision: 0.021315468940316686


Prediction accuracy: 0.9838183762441391


Recall score: 0.5198019801980198


ROC and AUC Score: 0.751964482162502


f1 score: 0.04095163806552262




In [0]:
# x_train_xg, x_val_xg, feature_xg  = feature_selection(XGBClassifier(random_state = rand_seed), 
# #                                           x_train, y_train, x_val, 10)
# xg_f1, xg_ctr = classifier(XGBClassifier(n_estimators = 500, max_depth = 5, learning_rate = 0.01, random_state = rand_seed), \
#                                              x_train_1, y_train, x_val_1, y_val)
# max_click_n_xg, c_optimal_n_xg, bid_price_final = non_linear(xg_ctr['click_proba'], validation)

In [0]:
# x_train_xg.to_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/train_1793_7172_xg_90.csv', sep = ',')
# x_val_xg.to_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/val_1793_7172_xg_90.csv', sep = ',')

In [0]:
# xg_ctr.to_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/xg_ctr_90.csv')

In [0]:
# features_xg = list(x_train_1.columns)

In [0]:
feature_xg.remove('payprice')

In [0]:
feature_xg.remove('bidprice')

In [0]:
len(feature_xg)

88

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_TEST_v1.csv')

In [0]:
group = pd.read_csv('/content/gdrive/My Drive/we_data/Group_xx.csv')

In [0]:
group.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,0.0
1,29167d4caa719788b5a342dbaa25151d53121f80,0.0
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,0.0
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,0.0
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,0.0


In [0]:
test = test[ feature_xg]

In [0]:
test.head()

,hour,slotprice,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,adexchange_1.0,...,slotsize_200*200,slotsize_300*250,slotsize_336*280,slotsize_360*300,slotsize_640*90,slotsize_728*90,slotsize_760*90,slotsize_910*90,slotsize_950*90,slotsize_960*90
0,12,10,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,14,5,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,0,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
3,21,30,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,20,50,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [0]:
x_train_xg = x_train_xg.drop(columns = ['bidprice', 'payprice'])

In [0]:
# test = test.as_matrix(columns = feature_xg)

model = XGBClassifier(n_estimators = 500, max_depth = 5, learning_rate = 0.01, random_state = rand_seed).fit(x_train_xg, y_train)
        # Predict on validation set
preds = model.predict(test)
        # Predicted probabilities
pred_proba = model.predict_proba(test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
pred_proba_1 = pd.DataFrame(pred_proba, columns = ['first', 'second'])

In [0]:
ctr_xg = pred_proba_1['second'] # 基于test

In [0]:
ctr_xg

In [0]:
# xg_ctr_90 = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/xg_ctr_90.csv')
# xg_ctr_90 = xg_ctr_90.drop('Unnamed: 0', axis = 1)
# xg_ctr_90 = xg_ctr_90['click_proba']

In [0]:
avg_ctr = ctr_xg.mean()

In [0]:
current_bid = 72.5 * ctr_xg/avg_ctr 

# c = 140
# lamda = 0.001
# ctr = ctr_xg
# current_bid = 1.25 * (c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
#                      -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3))))



In [0]:
def linear_2(ctr, validation):
  
  w = 7172/(2430981-1793)
  pctr_90 = xg_ctr_90/(xg_ctr_90 + (1- xg_ctr_90) / w)
  avgctr = 1793/2430981
  avgctr_2 = pctr_90.mean()
  
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  
  for base in range(80,130,1):
    current_bid = base * pctr_90  / avgctr_2
    budget = 6250*1000
    clicks = 0
    impressions = 0
   
    for j in range(len(current_bid)):
      payprice = validation.payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation.click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
          break
          
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
    print('base = {}, clicks = {}, impression = {}, budget = {}'.format(base, clicks,  impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  base_optimal = base_range[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when base = {}, budget = {}'.format(max_click, base_optimal, budget_max, impression_max))
  
  return max_click, c_optimal, bid_price_final

In [0]:
max_click, c_optimal, bid_price_final = linear_2(xg_ctr_90, validation)

In [0]:
'''
  pctr_90 = xg_ctr_90/(xg_ctr_90 + (1- xg_ctr_90) / w)
avgctr = 1793/2430981

base = 100, clicks = 160, impression = 97442, budget = 1735628
base = 110, clicks = 163, impression = 108794, budget = 1043010
base = 120, clicks = 167, impression = 120249, budget = 283602
base = 130, clicks = 152, impression = 121769, budget = 15

base = 119, clicks = 167, impression = 119142, budget = 359491
base = 120, clicks = 167, impression = 120249, budget = 283602
base = 121, clicks = 167, impression = 121430, budget = 208023
base = 122, clicks = 168, impression = 122625, budget = 127517
base = 123, clicks = 168, impression = 123803, budget = 45793
base = 124, clicks = 167, impression = 124240, budget = 13

 pctr_90 = xg_ctr_90/(xg_ctr_90 + (1- xg_ctr_90) / w)
 avgctr_1 = pctr_90.mean()
 
base = 86, clicks = 166, impression = 116322, budget = 552892
base = 87, clicks = 167, impression = 117896, budget = 445629
base = 88, clicks = 167, impression = 119317, budget = 347888
base = 89, clicks = 167, impression = 120825, budget = 246381
base = 90, clicks = 168, impression = 122478, budget = 136545
base = 91, clicks = 169, impression = 124087, budget = 25688
base = 92, clicks = 165, impression = 124027, budget = 53
base = 93, clicks = 161, impression = 123504, budget = 29
'''

In [0]:
w = 7172/(2430981-1793)
pctr = ctr_xg/(ctr_xg + (1- ctr_xg) / w)
avgctr = 1793/2430981
avgctr_1 = pctr_90.mean()

current_bid = 123 * pctr / avgctr

   

NameError: ignored

In [0]:
ctr_xg

In [0]:
group['bidprice'] = current_bid

In [0]:
group.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,82.630571
1,29167d4caa719788b5a342dbaa25151d53121f80,176.927449
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,29.504805
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,105.182913
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,69.060133


In [0]:
group['click'] = preds
group['prob'] = ctr_xg

In [0]:
group[(group['prob'] <= 0.4) & (group['prob'] >= 0.3) ]['bidprice']

In [0]:
group.to_csv('/content/gdrive/My Drive/we_data/SUBMISSION_BIDS/XG_linear_90_base72.5.csv')